In [1]:
import numpy as np
import pandas
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
dataframe = pandas.read_csv("SP500DATA.csv")
print(dataframe)# Text file data converted to integer data type

       Date    Price     Open     High      Low Vol.  Change %
0    Dec-21  4766.18  4623.80  4808.52  4494.20    -    0.0436
1    Nov-21  4567.00  4616.47  4743.74  4560.26    -   -0.0083
2    Oct-21  4605.38  4324.71  4608.70  4278.70    -    0.0691
3    Sep-21  4307.54  4531.04  4544.58  4304.90    -   -0.0476
4    Aug-21  4522.68  4415.90  4537.80  4369.20    -    0.0290
..      ...      ...      ...      ...      ...  ...       ...
618  Jun-70    72.72    76.55    79.96    72.25    -   -0.0500
619  May-70    76.55    81.52    82.32    68.61    -   -0.0610
620  Apr-70    81.52    89.63    90.70    79.31    -   -0.0905
621  Mar-70    89.63    89.50    91.07    86.19    -    0.0015
622  Feb-70    89.50    85.02    90.33    84.64    -    0.0527

[623 rows x 7 columns]


In [2]:
trainData = dataframe[["Price", "Open"]].to_numpy(dtype = 'float')
targetData = dataframe[["Price"]].to_numpy(dtype = 'float')

norm = MinMaxScaler(feature_range = (0, 1))
trainData = norm.fit_transform(trainData)
targetData = norm.fit_transform(targetData)

In [11]:
X = []
Y = []
for i in range(trainData.shape[0] - 5 - 4):
    X.append(trainData[i:i+6])
    Y.append(targetData[i+6:i+6+4])
X = np.array(X)
#X = np.flip(X)
Y = np.array(Y)
Y = np.swapaxes(Y, 1, 2)
#Y = np.flip(Y)
print(X.shape)
print(Y.shape)

(614, 6, 2)
(614, 1, 4)


In [4]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   

        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        hidden = self.init_hidden(batch_size)

        out, hidden = self.rnn(x, hidden)
        
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [5]:
RNN = Model(input_size=2, output_size=4, hidden_dim=16, n_layers=1)
optimizer = torch.optim.Adam(RNN.parameters(), lr=0.01)
criterion = nn.MSELoss()
RNN

Model(
  (rnn): RNN(2, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=4, bias=True)
)

In [12]:
#set seed for reproducible results 
loss_all_1 = []
for epoch in range(100):
    # make training sets in torch
    inputs = Variable(torch.from_numpy(X).float())
    labels = Variable(torch.from_numpy(Y).float())
    
    # zero the parameter (weight) gradients
    optimizer.zero_grad()

    # Perform forward pass
    output, hidden = RNN(inputs)   
    
    # Compute loss
    loss = criterion(output.reshape(-1), labels.view(-1))
    loss_all_1 += [loss.item()]
       
    # Backward pass
    loss.backward(retain_graph=True)
    
    if (epoch % 10 == 0 ):
        print("Epoch ", epoch, "MSE: ", loss.item())
    
    # Update parameters
    optimizer.step()                                

RuntimeError: The size of tensor a (14736) must match the size of tensor b (2456) at non-singleton dimension 0

In [10]:
xTensor = torch.from_numpy(X).float()
yTensor = torch.Tensor(Y).float()

for epoch in range(100):
    optimizer.zero_grad()
    output, hidden = RNN(xTensor)
    loss = criterion(output.view(-1), yTensor.view(-1))
    loss.backward()
    optimizer.step()
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, 100), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

C:\Users\Luka\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([2456])) that is different to the input size (torch.Size([14736])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (14736) must match the size of tensor b (2456) at non-singleton dimension 0